## Fill NUTS 3 regions

In this notebook we import the NUTS_3,2,1,0 regions where we performed zonal median analysis.
We want to fill empty NUTS_3 regions by going backwards up to NUTS_0 units, until we find a median value to use for such area. 
In case we do not find any valid value, we leave a nan number, (set to -999).

In [2]:
# list of imports
import pandas as pd
import os
# to read conf file
import configparser

In [ ]:
prj_path = r"E:\mottaco\Documents\Workspace\Drought_analysis\database\zonal_stats_NUTS3_tot.parquet"
prj_df_3 = pd.read_parquet(prj_path, engine='fastparquet')

In [ ]:
prj_df_3

In [ ]:
# import all NUTS level and set a multi-index
prj_path = r"E:\mottaco\Documents\Workspace\Drought_analysis\database\zonal_stats_NUTS1_tot.parquet"
prj_df_1 = pd.read_parquet(prj_path, engine='fastparquet').set_index(['model', 'rcp', 'year', 'NUTS_ID'])
prj_path = r"E:\mottaco\Documents\Workspace\Drought_analysis\database\zonal_stats_NUTS2_tot.parquet"
prj_df_2 = pd.read_parquet(prj_path, engine='fastparquet').set_index(['model', 'rcp', 'year', 'NUTS_ID'])
prj_path = r"E:\mottaco\Documents\Workspace\Drought_analysis\database\zonal_stats_NUTS0_tot.parquet"
prj_df_0 = pd.read_parquet(prj_path, engine='fastparquet').set_index(['model', 'rcp', 'year', 'NUTS_ID'])

In [ ]:
# Create new columns in NUTS-3 for all of them
prj_df_3["NUTS_ID_2"] = prj_df_3["NUTS_ID"].str[:-1]
prj_df_3["NUTS_ID_1"] = prj_df_3["NUTS_ID"].str[:-2]
prj_df_3["NUTS_ID_0"] = prj_df_3["NUTS_ID"].str[:-3]

In [ ]:
EU = ['AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','EL','HU','IE','IT','LV','LT','LU','MT','NL','PL','PT','RO','SK','SI','ES','SE','UK']

For DF3 we remove non EU countries

In [ ]:
prj_df_3 = prj_df_3[prj_df_3.NUTS_ID_0.isin(EU)]

Now we split in two dfs depending on med value

In [ ]:
# first adding a new column
mask = prj_df_3.med == -999
prj_df_3_OK = prj_df_3[~mask]
prj_df_3_NOT_OK = prj_df_3[mask]

In [ ]:
# add all dfs to a dict to simplify operations later on
db_dict = {
  0: prj_df_0,
  1: prj_df_1,
  2: prj_df_2,
  3: prj_df_3
}

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

Finally, we iterate over all rows and for each row we look recursively for a median value. Until we find one.

In [ ]:
max_count = len(prj_df_3_NOT_OK)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

# iterrate over ALL rows
for i, x in prj_df_3_NOT_OK.iterrows():
    f.value +=1
    nuts_level = 3
    while(nuts_level):
        to_subs = 4 - nuts_level
        nuts_level -= 1 
        new_med = db_dict[nuts_level].loc[(x.model,x.rcp,x.year, x.NUTS_ID[:-to_subs])].med
        if(new_med!=-999): break
    # bit slower than making a new column
    prj_df_3_NOT_OK.loc[i, 'med'] = new_med

# concatenate back and save
final_df = pd.concat([prj_df_3_OK,prj_df_3_NOT_OK]).reset_index(drop=True)
final_df.drop(['NUTS_ID_2', 'NUTS_ID_1','NUTS_ID_0'], axis=1,inplace=True)

In [ ]:
# finally we save to parquet
final_df.to_parquet('zonal_stats_NUTS{0}_filled_new.parquet'.format(3), engine='fastparquet')